# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [1]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [3]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [4]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [5]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

### * How many instructors taught at Brandeis last year?

In [7]:
instructors = len({c['instructor'] for c in courses if c['subject'] == 'COSI' })
instructors

27

### * Which terms are represented?

In [8]:
terms = {c['term'] for c in courses }
terms

{'1203', '1211', '1212'}

### * What is the total number of students taking COSI courses last year?

In [9]:
total = 0
for c in courses:
    if c['subject'] == 'COSI':
        total += c['enrolled']
total

2223

### * What was the median size of a COSI course last year (counting only those courses with at least 10 students)

In [10]:
median = []
for c in courses:
    if c['subject'] == 'COSI' and c['enrolled'] >= 1:
        median.append(c['enrolled'])
np.median(median)

2.0

### * Create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.

In [11]:
student_list = {}
for c in courses:
    if c['subject'] in student_list:
        student_list[c['subject']] += c['enrolled']
    else:
        student_list[c['subject']] = c['enrolled']
students_courses = []
for subject, num in student_list.items():
    students_courses.append((subject, num))
students_courses.sort(key=lambda x: x[1], reverse=True)
students_courses[0:10]

[('HS', 5318),
 ('BIOL', 3085),
 ('BUS', 2766),
 ('HWL', 2734),
 ('CHEM', 2322),
 ('ECON', 2315),
 ('COSI', 2223),
 ('MATH', 1785),
 ('PSYC', 1704),
 ('ANTH', 1144)]

### * Do the same as in (d) but print the top 10 subjects in terms of number of courses offered

In [12]:
subjects = defaultdict(set)
for c in courses:
    subjects[c['subject']].add(c['name'])

subjects_course_count = []
for subject, course in subjects.items():
    subjects_course_count.append((subject, len(course)))
subjects_course_count.sort(key=lambda x: x[1], reverse=True)
subjects_course_count[0:10]

[('HS', 170),
 ('ENG', 67),
 ('BIOL', 62),
 ('BUS', 58),
 ('ANTH', 56),
 ('MUS', 56),
 ('PSYC', 55),
 ('MATH', 53),
 ('ECON', 49),
 ('FA', 49)]

### * Do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject

In [13]:
faculty = defaultdict(set)
for c in courses:
    faculty[c['subject']].add(c['instructor'][2])

faculty_count = []
for subject, instructor in faculty.items():
    faculty_count.append((subject, (len(instructor))))
faculty_count.sort(key=lambda x: x[1], reverse=True)
faculty_count[0:10]


[('HS', 84),
 ('BIOL', 67),
 ('ECON', 52),
 ('BCHM', 49),
 ('HIST', 47),
 ('BUS', 46),
 ('BCBP', 46),
 ('HWL', 42),
 ('MATH', 37),
 ('NEJS', 37)]

### * What were the five largest courses (where we combine sections)?

In [14]:
combined_section_courses = {}
for c in courses:
    if (c['subject'], c['coursenum']) in combined_section_courses:
        combined_section_courses[(c['subject'], c['coursenum'])] = max(c['enrolled'], combined_section_courses[(c['subject'], c['coursenum'])]) 
    else:
        combined_section_courses[(c['subject'], c['coursenum'])] = c['enrolled']
combined_section_number = []
for course, num in combined_section_courses.items():
    combined_section_number.append((course, num))
combined_section_number.sort(key=lambda x: x[1], reverse=True)
combined_section_number[0:5]

[(('HWL', '1-PRE'), 784),
 (('CHEM', '25A'), 186),
 (('BIOL', '42A'), 186),
 (('BIOL', '15B'), 181),
 (('CHEM', '25B'), 180)]

### * List the top 20 faculty in terms of number of students they taught

In [15]:
faculty_students = {}
for c in courses:
    if c['instructor'][0:2] in faculty_students:
        faculty_students[c['instructor'][0:2]] += c['enrolled']
    else:
        faculty_students[c['instructor'][0:2]] = c['enrolled']
students_faculty_count = []
for faculty, num in faculty_students.items():
    students_faculty_count.append((faculty, num))
students_faculty_count.sort(key=lambda x: x[1], reverse=True)
students_faculty_count[0:20]

[(('Leah', 'Berkenwald'), 926),
 (('Kene Nathan', 'Piasta'), 583),
 (('Stephanie', 'Murray'), 515),
 (('Milos', 'Dolnik'), 489),
 (('Maria', 'de Boef Miara'), 450),
 (('Bryan', 'Ingoglia'), 439),
 (('Rachel V.E.', 'Woodruff'), 422),
 (('Timothy J', 'Hickey'), 411),
 (('Daniel', 'Breen'), 375),
 (('Melissa', 'Kosinski-Collins'), 365),
 (('Claudia', 'Novack'), 355),
 (('Antonella', 'DiLillo'), 342),
 (('Jon', 'Chilingerian'), 330),
 (('Ahmad', 'Namini'), 327),
 (('Iraklis', 'Tsekourakis'), 316),
 (('Geoffrey', 'Clarke'), 315),
 (('Peter', 'Mistark'), 277),
 (('Brenda', 'Anderson'), 275),
 (('Colleen', 'Hitchcock'), 261),
 (('Scott A.', 'Redenius'), 259)]

### * list the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number)

In [16]:
courses_enrolled = {}
for c in courses:
    if (c['subject'], c['coursenum']) in courses_enrolled:
        courses_enrolled[(c['subject'], c['coursenum'])] = max(c['enrolled'],courses_enrolled[(c['subject'], c['coursenum'])])
    else:
        courses_enrolled[(c['subject'], c['coursenum'])] = c['enrolled']
arr_courses_enrolled = []
for course, num in courses_enrolled.items():
    arr_courses_enrolled.append((course, num))
arr_courses_enrolled.sort(key=lambda x: x[1], reverse=True)
arr_courses_enrolled[0:20]

[(('HWL', '1-PRE'), 784),
 (('CHEM', '25A'), 186),
 (('BIOL', '42A'), 186),
 (('BIOL', '15B'), 181),
 (('CHEM', '25B'), 180),
 (('HS', '104B'), 175),
 (('PSYC', '10A'), 170),
 (('COSI', '164A'), 166),
 (('COSI', '10A'), 150),
 (('LGLS', '10A'), 145),
 (('ENVS', '2A'), 141),
 (('BIOL', '16A'), 136),
 (('NPSY', '199A'), 135),
 (('PSYC', '34B'), 119),
 (('PHYS', '10A'), 109),
 (('PHYS', '10B'), 107),
 (('BIOL', '350A'), 105),
 (('PSYC', '51A'), 105),
 (('ECON', '10A'), 104),
 (('SOC', '1A'), 103)]

### * What are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)

In [ ]:
subjects = {c['subject'] for c in courses }
print(subjects)

### * Which terms are represented?

In [ ]:
terms = {c['term'] for c in courses }
terms